# Setting

In [1]:
import sys, os

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
sys.path.append("../../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import AD_api
from Clust.clust.ML.tool import meta

db_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

cpu is available.


# 1. set param from Front End

In [2]:
modelName = 'anomaly_detection_SWaT_anomaly_detection_BeatGAN_ad_'
# feature list는 model_info에서 받아와야 합니다.
param1 = {
    'model_name': modelName,
    'ingestion_param_y':{
        "feature_list": "None"
    }
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = param1

# chage tpye string to bool -> ex) 'true' -> True
params = AD_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
#model_meta_db_style ='document'

# for testing mongodb model
#model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)
# model_meta = meta.read_model_meta_by_db_style(model_meta_db_style, meta_file_name, model_name)

# scaler info for inference data preparation
params['scaler_param'] = model_meta['scaler_param']

# model & target data info for ML inference
params['model_info'] = model_meta['model_info']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']

In [5]:
params['model_info']['train_parameter']['batch_size'] = 1

In [6]:
# for data preparation (temp)
def pre_infer_df(infer_df):
    infer_x = infer_df.drop(columns=['timestamp','attack'])

    return infer_x

# 3. Data Preparation
# make inference data X, scaler y
# infer_X, scaler_y = AD_api.infer_data_preparation(params, infer_np)
scaler_y = None

# 추후 ML_api 의 데이터 준비 형식으로 대체 가능
import pandas as pd

# from test.csv
# 보통 train set 에서 train/val 나누지 않는지?
infer_df = pd.read_csv('./data/inference_ver2.csv')
infer_X = pre_infer_df(infer_df)

In [7]:
print(type(infer_X))
print(infer_X.shape)  # TBD: [seq_len, n_features]

<class 'pandas.core.frame.DataFrame'>
(500, 51)


In [8]:
# model inference
prediction_result = AD_api.AD_inference(params, infer_X, scaler_y)

(500, 51)

Start inference

torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])
torch.Size([1, 51, 320])


In [9]:
print(type(prediction_result))
print(prediction_result)

<class 'pandas.core.frame.DataFrame'>
     value
0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
..     ...
175    1.0
176    1.0
177    1.0
178    1.0
179    1.0

[180 rows x 1 columns]
